In [1]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model


In [2]:
from federatedscope.contrib.data.cikm_cup import call_cikm_cup_data

register_data("cikm_cup", call_cikm_cup_data)

### Register model

In [3]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [4]:
from federatedscope.contrib.trainer.laplacian_trainer import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

/home/michael/Projects/CKIM_other/CIKM22_FL_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/michael/Projects/CKIM_other/CIKM22_FL_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "


 89.1### Register metric

### Create the config

#### Set data, model, trainer and metric

In [5]:
from federatedscope.core.configs.config import global_cfg, CN
cfg = global_cfg.clone()

cfg.merge_from_file("federatedscope/gfl/baseline/laplacian_gine_on_cikmcup.yaml")
#cfg.data.type = 'cikm_cup'
#cfg.data.root = 'data'
#cfg.data.shuffle=True
#cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.], 'std': [1]}]]
#cfg.model.type = 'gin'
#cfg.model.out_channels = 10
#cfg.model.hidden = 64
#cfg.model.task='graph'
#cfg.model.dropout = 0.5
#cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'clf']
#cfg.train.batch_or_epoch = "epoch"
#cfg.trainer.type = 'laplacian_trainer'
#cfg.data.batch_size = 64
# cfg.eval.metric = ['mymetric']

cfg.params = CN()
cfg.params.alpha=0.1
cfg.params.csd_importance=1e2
cfg.params.eps=1e-15
cfg.params.p=0.

#### configure other options

In [6]:
#cfg.use_gpu = True
#cfg.best_res_update_round_wise_key = "test_loss"

#cfg.federate.mode = 'standalone'
cfg.federate.method = \
    'Laplacian_batch_all_clients_csd_1e3_new'
#cfg.federate.local_update_steps = 20000000
#cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
#cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 20000

cfg.federate.client_num = 13
cfg.early_stop.patience = 20000
#cfg.train.optimizer.lr = 0.001
#cfg.train.optimizer.weight_decay = 0.0005
#cfg.grad.grad_clip = 5.0
#cfg.criterion.type = 'CrossEntropyLoss'
#cfg.seed = 123
cfg.eval.freq = 1
cfg.eval.metrics = ['imp_ratio']
cfg.eval.report = ['avg']
cfg.eval.best_res_update_round_wise_key = 'val_imp_ratio'
cfg.eval.count_flops = False


In [7]:
import torch
#torch.manual_seed(0)
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(False)
#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

In [8]:
from yacs.config import CfgNode
client_cfg_file = "federatedscope/gfl/baseline/laplacian_gine_on_cikmcup_per_client.yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [9]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


2022-11-28 19:47:26,681 (trainer_builder:11)WARNING: No module named 'federatedscope.contrib.optimizer' in `federatedscope.contrib.trainer`, some modules are not available.


In [10]:
from federatedscope.contrib.workers.laplacian_client import LaplacianClient
from federatedscope.contrib.workers.laplacian_server import LaplacianServer


setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


2022-11-28 19:47:26,693 (utils:129)INFO: the current machine is at 127.0.1.1
2022-11-28 19:47:26,694 (utils:131)INFO: the current dir is /home/michael/Projects/CKIM_Competition
2022-11-28 19:47:26,695 (utils:132)INFO: the output dir is exp/Laplacian_batch_all_clients_csd_1e2_new_gin_on_cikmcup_lr0.1_lstep10_/sub_exp_20221128194726
2022-11-28 19:47:26,758 (cikm_cup:57)INFO: Loading CIKMCUP data from /home/michael/Projects/CKIM_Competition/data/CIKM22Competition.
2022-11-28 19:47:26,760 (cikm_cup:67)INFO: Loading CIKMCUP data for Client #1.
2022-11-28 19:47:27,133 (cikm_cup:67)INFO: Loading CIKMCUP data for Client #2.
2022-11-28 19:47:27,170 (cikm_cup:67)INFO: Loading CIKMCUP data for Client #3.
2022-11-28 19:47:27,600 (cikm_cup:67)INFO: Loading CIKMCUP data for Client #4.
2022-11-28 19:47:27,623 (cikm_cup:67)INFO: Loading CIKMCUP data for Client #5.
2022-11-28 19:47:27,669 (cikm_cup:67)INFO: Loading CIKMCUP data for Client #6.
2022-11-28 19:47:28,029 (cikm_cup:67)INFO: Loading CIKMCUP d

In [ ]:
Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServer,
                       client_class=LaplacianClient,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
server params: 
encoder_atom.atom_embedding_list.0.weight


2022-11-28 19:47:30,509 (fed_runner:249)INFO: Server #0 has been set up ... 
2022-11-28 19:47:30,526 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 8
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
    min_curv: 0.02
  root: data/
  server_holds_all: False
  shuffle: True
  sizes: [10, 5]
  splits: [0.8, 0.1, 0.1]
  splitter: 
  splitter_args: []
  subsample: 1.0
  target_transform: []
  transform: []
  type: 

encoder_atom.atom_embedding_list.1.weight
encoder_atom.atom_embedding_list.2.weight
encoder_atom.atom_embedding_list.3.weight
encoder_atom.atom_embedding_list.4.weight
encoder_atom.atom_embedding_list.5.weight
encoder_atom.atom_embedding_list.6.weight
encoder_atom.atom_embedding_list.7.weight
encoder_atom.atom_embedding_list.8.weight
encoder_atom.atom_embedding_list.9.weight
encoder_atom.atom_embedding_list.10.weight
encoder_atom.atom_embedding_list.11.weight
encoder_atom.atom_embedding_list.12.weight
encoder_atom.atom_embedding_list.13.weight
encoder_atom.atom_embedding_list.14.weight
encoder_atom.atom_embedding_list.15.weight
encoder_atom.atom_embedding_list.16.weight
encoder_atom.atom_embedding_list.17.weight
encoder_atom.atom_embedding_list.18.weight
encoder_atom.atom_embedding_list.19.weight
encoder_atom.atom_embedding_list.20.weight
encoder_atom.atom_embedding_list.21.weight
encoder.weight
encoder.bias
gnn.convs.0.nn.linears.0.weight
gnn.convs.0.nn.linears.0.bias
gnn.convs.0.nn.l

2022-11-28 19:47:30,707 (trainer:50)WARNING: Will not use monitor in trainer with class <class 'federatedscope.contrib.trainer.laplacian_trainer.LaplacianTrainer'>
2022-11-28 19:47:30,708 (fed_runner:302)INFO: Client 2 has been set up ... 
2022-11-28 19:47:30,861 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 8
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
    min_curv: 0.02
  root: data/
  server_holds_all:

track_running_stats: True
track_running_stats: True
track_running_stats: True


...

2022-11-28 19:47:31,154 (trainer:50)WARNING: Will not use monitor in trainer with class <class 'federatedscope.contrib.trainer.laplacian_trainer.LaplacianTrainer'>
2022-11-28 19:47:31,155 (fed_runner:302)INFO: Client 5 has been set up ... 
2022-11-28 19:47:31,171 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 8
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
    min_curv: 0.02
  root: data/
  server_holds_all:

track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True


2022-11-28 19:47:31,371 (trainer:50)WARNING: Will not use monitor in trainer with class <class 'federatedscope.contrib.trainer.laplacian_trainer.LaplacianTrainer'>
2022-11-28 19:47:31,375 (fed_runner:302)INFO: Client 7 has been set up ... 
2022-11-28 19:47:31,398 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 8
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
    min_curv: 0.02
  root: data/
  server_holds_all:

track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True


2022-11-28 19:47:46,941 (laplacian_client:118)INFO: {'Role': 'Client #7', 'Round': 0, 'Results_raw': {'train_imp_ratio': 27.994462, 'train_total': 22280, 'train_loss': 10372.301694, 'train_avg_loss': 0.465543}}
2022-11-28 19:48:02,268 (laplacian_client:118)INFO: {'Role': 'Client #3', 'Round': 0, 'Results_raw': {'train_imp_ratio': 21.878316, 'train_total': 22190, 'train_loss': 12236.036578, 'train_avg_loss': 0.551421}}
2022-11-28 19:48:03,623 (laplacian_client:118)INFO: {'Role': 'Client #2', 'Round': 0, 'Results_raw': {'train_imp_ratio': -56.426824, 'train_total': 1810, 'train_loss': 1329.480715, 'train_avg_loss': 0.73452}}
2022-11-28 19:48:09,009 (laplacian_client:118)INFO: {'Role': 'Client #8', 'Round': 0, 'Results_raw': {'train_imp_ratio': 19.995145, 'train_total': 7770, 'train_loss': 3230.110096, 'train_avg_loss': 0.415716}}
2022-11-28 19:48:09,867 (laplacian_client:118)INFO: {'Role': 'Client #4', 'Round': 0, 'Results_raw': {'train_imp_ratio': -57.656398, 'train_total': 1010, 'train

4

In [ ]:
data

In [ ]:
import os

In [ ]:
os.environ['CUBLAS_WORKSPACE_CONFIG']